<a href="https://colab.research.google.com/github/tonykks/my-first-web-site/blob/main/2%EC%A7%84%EC%88%98%EC%9D%98_%EC%82%AC%EC%B9%99%EC%97%B0%EC%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
def to_binary(n):
    return format(n, '04b')

def from_binary(b):
    return int(b, 2)

def add_binary(a, b):
    max_len = max(len(a), len(b))
    a = a.zfill(max_len)
    b = b.zfill(max_len)

    result = ''
    carry = 0

    for i in range(max_len - 1, -1, -1):
        r = carry
        r += 1 if a[i] == '1' else 0
        r += 1 if b[i] == '1' else 0
        result = ('1' if r % 2 == 1 else '0') + result
        carry = 0 if r < 2 else 1

    if carry != 0:
        result = '1' + result

    return result[-4:], carry

def twos_complement(b):
    n = len(b)
    ones_complement = ''.join('1' if bit == '0' else '0' for bit in b)
    return add_binary(ones_complement, '1'.zfill(n))[0]

def subtract_binary(a, b):
    b_twos_complement = twos_complement(b)
    result, carry = add_binary(a, b_twos_complement)
    return result[-4:], carry

def multiply_binary(a, b):
    a_dec = from_binary(a)
    b_dec = from_binary(b)
    result_dec = 0
    steps = []

    for _ in range(b_dec):
        prev_result_dec = result_dec
        result_dec += a_dec
        steps.append((prev_result_dec, a_dec, result_dec))

    result_bin = to_binary(result_dec & 0b1111)
    steps_bin = [(to_binary(prev & 0b1111), to_binary(a_dec & 0b1111), to_binary(res & 0b1111)) for prev, a_dec, res in steps]
    return result_bin, steps_bin

def divide_binary(a, b):
    a_dec = from_binary(a)
    b_dec = from_binary(b)
    if b_dec == 0:
        raise ValueError("Cannot divide by zero.")

    result_dec = 0
    remainder = a_dec
    steps = []

    while remainder >= b_dec:
        prev_remainder = remainder
        b_1s_complement = ''.join('1' if bit == '0' else '0' for bit in to_binary(b_dec))
        b_twos_complement = twos_complement(to_binary(b_dec))
        remainder_bin, _ = add_binary(to_binary(remainder & 0b1111), b_twos_complement)
        remainder = from_binary(remainder_bin)
        result_dec += 1
        steps.append((to_binary(prev_remainder & 0b1111), b_1s_complement, b_twos_complement, remainder_bin, to_binary(result_dec & 0b1111)))

    result_bin = to_binary(result_dec & 0b1111)
    return result_bin, steps

def print_calculation(expression):
    a, operator, b = expression.split()
    a_dec = int(a)
    b_dec = int(b)
    a_bin = to_binary(a_dec)
    b_bin = to_binary(b_dec)

    print(f"{a}를 2진수로 표현: {a_bin}")
    print(f"{b}를 2진수로 표현: {b_bin}")

    if operator == '+':
        result_bin, _ = add_binary(a_bin, b_bin)
        print(f"덧셈 수행: {a_bin} + {b_bin}")
        print(f"  {a_bin}")
        print(f"+ {b_bin}")
        print(f"-------")
        print(f"  {result_bin}")
    elif operator == '-':
        b_1s_complement = ''.join('1' if bit == '0' else '0' for bit in b_bin)
        b_2s_complement = twos_complement(b_bin)
        result_bin, carry = subtract_binary(a_bin, b_bin)
        print(f"2의 보수 구하기:")
        print(f"{b}의 1의 보수: {b_1s_complement}")
        print(f"{b}의 2의 보수: {b_1s_complement} + 0001 = {b_2s_complement}")
        print(f"덧셈으로 뺄셈 수행:")
        print(f"{a_bin} + {b_2s_complement}")
        print(f"  {a_bin}")
        print(f"+ {b_2s_complement}")
        print(f"-------")
        print(f"  {result_bin} (올림수 {carry} 버림)")
    elif operator == '*':
        result_bin, steps_bin = multiply_binary(a_bin, b_bin)
        print(f"곱셈 수행: {a_bin} * {b_bin}")
        for prev, adder, res in steps_bin:
            print(f"  {prev} + {adder} = {res}")
    elif operator == '/':
        result_bin, steps = divide_binary(a_bin, b_bin)
        print(f"{a}에서 {b}을 계속해서 빼는 계산 수행")
        print(f"-{b}의 2의 보수는 {steps[0][1]} + 0001 = {steps[0][2]}")
        for i, (prev_remainder, b_1s_complement, b_twos_complement, remainder, quotient) in enumerate(steps):
            print(f"{prev_remainder} - {b_bin} = {prev_remainder} + (-{b_bin}) :")
            print(f"  {prev_remainder}")
            print(f"+ {b_twos_complement}")
            print(f"  {'-' * 7}")
            print(f"  {remainder}({from_binary(remainder)}) (올림수 1 버림) : {i + 1}번(몫: {quotient})")
    else:
        raise ValueError("Unknown operator: " + operator)

    result_dec = from_binary(result_bin)
    print(f"결과: {result_bin} (2진수), {result_dec} (10진수)")

# 예제 사용:
while True:
    expression = input("수식을 입력하세요 (예: 8 + 7, Enter = 종료): ")
    if expression == "":
        print("프로그램을 종료합니다.")
        break
    print("\n======================")
    print(f"Expression: {expression}")
    print_calculation(expression)
    print("======================\n")


수식을 입력하세요 (예: 8 + 7, Enter = 종료): 5 + 7

Expression: 5 + 7
5를 2진수로 표현: 0101
7를 2진수로 표현: 0111
덧셈 수행: 0101 + 0111
  0101
+ 0111
-------
  1100
결과: 1100 (2진수), 12 (10진수)

수식을 입력하세요 (예: 8 + 7, Enter = 종료): 7 - 5

Expression: 7 - 5
7를 2진수로 표현: 0111
5를 2진수로 표현: 0101
2의 보수 구하기:
5의 1의 보수: 1010
5의 2의 보수: 1010 + 0001 = 1011
덧셈으로 뺄셈 수행:
0111 + 1011
  0111
+ 1011
-------
  0010 (올림수 1 버림)
결과: 0010 (2진수), 2 (10진수)

수식을 입력하세요 (예: 8 + 7, Enter = 종료): 5 * 3

Expression: 5 * 3
5를 2진수로 표현: 0101
3를 2진수로 표현: 0011
곱셈 수행: 0101 * 0011
  0000 + 0101 = 0101
  0101 + 0101 = 1010
  1010 + 0101 = 1111
결과: 1111 (2진수), 15 (10진수)

수식을 입력하세요 (예: 8 + 7, Enter = 종료): 8 / 2

Expression: 8 / 2
8를 2진수로 표현: 1000
2를 2진수로 표현: 0010
8에서 2을 계속해서 빼는 계산 수행
-2의 2의 보수는 1101 + 0001 = 1110
1000 - 0010 = 1000 + (-0010) :
  1000
+ 1110
  -------
  0110(6) (올림수 1 버림) : 1번(몫: 0001)
0110 - 0010 = 0110 + (-0010) :
  0110
+ 1110
  -------
  0100(4) (올림수 1 버림) : 2번(몫: 0010)
0100 - 0010 = 0100 + (-0010) :
  0100
+ 1110
  -------
  0010(2) (올림